### Урок 4. Парсинг HTML. XPath

#### Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:

название источника;<br>
наименование новости;<br>
ссылку на новость;<br>
дата публикации.

In [1]:
from lxml import html
from pprint import pprint
import requests
import re

#### news.mail.ru

In [2]:
link_mail = 'https://news.mail.ru'
header = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}


req = requests.get(link_mail + '/politics/', headers=header)

dom = html.fromstring(req.text)

news = []

items = dom.xpath(".//div[@data-counter-id='20265693']//a")

for item in items:
    b = {}

    title = item.xpath(".//span[@class='photo__title']/text()")
    link = item.xpath(".//@href")
    # проваливаемся в статьи
    a_req = requests.get(link[0], headers=header)
    a_dom = html.fromstring(a_req.text)
    source = a_dom.xpath(".//span[contains(text(),'источник')]/..//a/span/text()")
    date = a_dom.xpath(".//span[@class='note__text breadcrumbs__text js-ago']/@datetime")
    date = re.split(r'T|\+', date[0])[0:2]
    date = ' '.join(date)

    b['title'] = title
    b['link'] = link
    b['source'] = source
    b['date'] = date

    news.append(b)


items = dom.xpath("//div[@class='paging__content js-pgng_cont']/div[contains(@class,'newsitem')]")

for item in items:
    b = {}
    link = item.xpath(".//a[@class='newsitem__title link-holder']/@href")
    title = item.xpath(".//a[@class='newsitem__title link-holder']/span/text()")[0].replace(u'\xa0', u' ')
    date = item.xpath(".//span[@class='newsitem__param js-ago']/@datetime")
    date = re.split(r'T|\+', date[0])[0:2]
    date = ' '.join(date)
    source = item.xpath(".//span[@class='newsitem__param']/text()")

    b['link'] = link
    b['title'] = title
    b['date'] = date
    b['source'] = source

    news.append(b)

pprint(news)

[{'date': '2020-09-13 16:18:46',
  'link': ['https://news.mail.ru/politics/43345119/'],
  'source': ['Коммерсантъ'],
  'title': ['Канцлер Австрии сообщил о начале второй волны коронавируса в '
            'стране']},
 {'date': '2020-09-13 15:44:24',
  'link': ['https://news.mail.ru/politics/43344832/'],
  'source': ['РИА Новости'],
  'title': ['МВД Белоруссии сообщило о задержании 250 человек в Минске']},
 {'date': '2020-09-13 13:32:46',
  'link': ['https://news.mail.ru/politics/43343695/'],
  'source': ['РБК'],
  'title': ['Памфилова заявила о возможности для ЦИК «поучиться» у '
            'Казахстана']},
 {'date': '2020-09-13 13:17:25',
  'link': ['https://news.mail.ru/politics/43343536/'],
  'source': ['Lenta.Ru'],
  'title': ['Белорусские хакеры пригрозили Лукашенко обрушением налоговой '
            'системы']},
 {'date': '2020-09-13 12:47:53',
  'link': ['https://news.mail.ru/politics/43343261/'],
  'source': ['Lenta.Ru'],
  'title': 'Названы регионы России с самой высокой и низ

#### lenta.ru

In [3]:
link_lenta = 'https://lenta.ru'
header = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}


req = requests.get(link_lenta + '/rubrics/russia/regions/', headers=header)

dom = html.fromstring(req.text)

news = []
items = dom.xpath("//div[@class='news']")
for item in items:
    b = {}
    link = [link_lenta + item.xpath(".//h4/a/@href | .//div[@class='b-feature__header']/a/@href")[0]]
    title = item.xpath(".//h4/a/text() | .//div[@class='b-feature__header']/a/text()")[0].replace(u'\xa0', u' ')
    date = item.xpath(".//div[@class='g-date']/text() | .//div[@class='g-date']/span[@class='time']/text() | .//span[@class='g-time']/text()")
    date = ' '.join(date)
    b['link'] = link
    b['title'] = title
    b['date'] = date
    b['source'] = 'Lenta.ru'

    news.append(b)


pprint(news)

[{'date': '15:33',
  'link': ['https://lenta.ru/news/2020/09/12/no_alcohol/'],
  'source': 'Lenta.ru',
  'title': 'В российском селе запретили продавать алкоголь'},
 {'date': '00:10 12 сентября',
  'link': ['https://lenta.ru/news/2020/09/11/region/'],
  'source': 'Lenta.ru',
  'title': 'Российскую вакцину от коронавируса направили в регионы'},
 {'date': '23:49 11 сентября',
  'link': ['https://lenta.ru/news/2020/09/11/pilot/'],
  'source': 'Lenta.ru',
  'title': 'При жесткой посадке самолета в российском регионе погиб пилот'},
 {'date': '17:33 11 сентября',
  'link': ['https://lenta.ru/news/2020/09/11/congratulation/'],
  'source': 'Lenta.ru',
  'title': 'Воробьев наградил сотрудников многофункциональных центров '
           'Подмосковья'},
 {'date': '14:56 11 сентября',
  'link': ['https://lenta.ru/news/2020/09/11/mascarilla/'],
  'source': 'Lenta.ru',
  'title': 'Подмосковное предприятие поставит медицинские маски в Европу'},
 {'date': '14:54 11 сентября',
  'link': ['https://lenta.r

#### yandex-новости

In [4]:
link_yandex = 'https://yandex.ru'
header = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'}


req = requests.get(link_yandex + '/news/rubric/politics', headers=header)

dom = html.fromstring(req.text)

news = []

items = dom.xpath("//div[@class='mg-grid__col mg-grid__col_xs_8'] | //article")

for item in items:
    b = {}
    link = item.xpath(".//a[@class='news-card__link']/@href")
    title = item.xpath(".//h2[@class='news-card__title']/text()")
    date = item.xpath(".//span[@class='mg-card-source__time']/text()")
    source = item.xpath(".//span[@class='mg-card-source__source']/a/text()")

    b['link'] = link
    b['title'] = title
    b['date'] = date
    b['source'] = source

    news.append(b)

pprint(news)

[{'date': ['17:25'],
  'link': ['https://yandex.ru/news/story/Bloomberg_Putin_ne_verit_chto_oppoziciya_smestit_Lukashenko--df656a4858bb03b193ff098083bf5c46?lang=ru&rubric=politics&stid=QVb5rchEQBdDjmC23-a1&t=1600007430&tt=true&persistent_id=112323242'],
  'source': ['Коммерсантъ'],
  'title': ['Bloomberg: Путин не верит, что оппозиция сместит Лукашенко']},
 {'date': ['17:25'],
  'link': ['https://yandex.ru/news/story/Bloomberg_Putin_ne_verit_chto_oppoziciya_smestit_Lukashenko--df656a4858bb03b193ff098083bf5c46?lang=ru&rubric=politics&stid=QVb5rchEQBdDjmC23-a1&t=1600007430&tt=true&persistent_id=112323242'],
  'source': ['Коммерсантъ'],
  'title': ['Bloomberg: Путин не верит, что оппозиция сместит Лукашенко']},
 {'date': ['17:25'],
  'link': ['https://yandex.ru/news/story/V_Germanii_nashli_varianty_zapreta_Severnogo_potoka-2_chuzhimi_rukami--0e4e71fc93097b96168d393874775ce9?lang=ru&rubric=politics&stid=7vmPKiO9OlR4DnD6G3Aa&t=1600007430&tt=true&persistent_id=112308752'],
  'source': ['Вест